In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import tensorflow as tf

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense, Activation, Conv1D, Flatten, Dropout, MaxPooling1D

import sys
sys.path.append('../../..')
from smartFAQ.src.tokenPad import tokenization_padding

import os

print('Setup is ready')

Setup is ready


Using TensorFlow backend.


# Load Data
---
## Fast Text model

In [2]:
#with open('../../data/fast_text_model_pretrained_weights.pickle', 'rb') as handle:
#    fast_text_model = pickle.load(handle)
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

In [3]:
#pretrained_weights = fast_text_model.wv.syn0
#vocab_size, embedding_size = pretrained_weights.shape

## Tokenization & Padding

In [4]:
MAX_NUM_WORDS = 20000


In [5]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_train['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_train['answer'])

fast_text_idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(fast_text_idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 30369
Length of longest sentence in input: 1688


In [6]:
x_train_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)

print("encoder_input_sequences.shape:", x_train_pad.shape)
print("encoder_input_sequences[172]:", x_train_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [  0   0   0 ... 135 183  86]


In [7]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(X_val['answer'])
input_integer_seq = input_tokenizer.texts_to_sequences(X_val['answer'])
x_val_pad = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", x_val_pad.shape)
print("encoder_input_sequences[172]:", x_val_pad[72])

encoder_input_sequences.shape: (10000, 1688)
encoder_input_sequences[172]: [   0    0    0 ... 5837 5419 5838]


## Keras Model

In [8]:
import gensim
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser

In [9]:
# sent = []
# for col in ['question', 'answer']:
#     words = [row.split() for row in X_train[col]]
#     sent = sent + words
# sentences = [[word for word in document.lower().split()] for document in X_train['answer']]

In [10]:
#sentences = [[word for word in document.lower().split()] for document in X_train['answer']]

In [11]:
#phrases = Phrases(sentences)
#bigram = Phraser(phrases)
#sentences = bigram[sentences]

In [12]:
# fast_text_model = FastText(min_count=50, window=10, size=300, sample=6e-5, alpha=0.03, min_alpha=0.0007, negative=50, workers=cores-1)

In [13]:
# sentences

In [14]:
with open('../../data/fast_text_model_pretrained_weights.pickle', 'rb') as handle:
    fast_text_model = pickle.load(handle)

In [15]:
pretrained_weights = fast_text_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

/Users/axelchenu/Work_directory/DATA_SCIENCE_YNOV/SMART_FAQ/smartFAQ/envs/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [16]:
#fast_text_model.most_similar(['import'])

In [17]:
x_train_pad.shape

(10000, 1688)

In [18]:
X_train['answer'].iloc[0]

'You use list comprehension rebuild list indices newlist v v enumerate oldlist indices I used set indices set membership testing faster list Note delete best done del lst index partially rebuilds list well one loop list comprehension efficient Demo gt gt gt oldlist gt gt gt indices gt gt gt v v enumerate oldlist indices'

In [19]:
fast_text_model.wv[X_train['answer'].iloc[0]]

array([ 0.5499138 ,  0.14493077,  0.06994156,  0.34921375, -0.02642111,
       -0.0097463 ,  0.45093492, -0.20186432, -0.30533472, -0.21423948,
        0.13659018,  0.20149484,  0.09093372, -0.0784286 , -0.45261833,
        0.17564005,  0.11498763,  0.10608989,  0.22079894, -0.02017957,
        0.3850254 ,  0.03616932, -0.3220436 , -0.28442326,  0.3687296 ,
       -0.04920127,  0.14371724, -0.0987426 ,  0.08710776,  0.04617305,
       -0.16039702,  0.21675728, -0.29762214,  0.17865299,  0.15160161,
       -0.1965639 ,  0.06691138,  0.08384872, -0.05414282,  0.03437477,
        0.12081204, -0.10925934, -0.16254102,  0.24165435,  0.1095176 ,
        0.12773256,  0.01760187,  0.20758538, -0.00531439, -0.34692603,
        0.25210643,  0.14800027,  0.17874879,  0.00872796, -0.16992536,
        0.06981291, -0.15173166,  0.05147651, -0.20842004,  0.1095694 ,
        0.10821486, -0.11381464, -0.01745203,  0.06112472, -0.3329582 ,
        0.12596047, -0.00137429, -0.07155851, -0.29180777,  0.13

In [20]:
matrix = np.zeros((X_train.shape[0], 300))
for i in range(X_train.shape[0]):
    matrix[i] = fast_text_model.wv[X_train['answer'].iloc[i]]
    

In [21]:
matrix_val = np.zeros((X_val.shape[0], 300))
for i in range(X_val.shape[0]):
    matrix_val[i] = fast_text_model.wv[X_val['answer'].iloc[i]]

In [22]:
X_train_vec = matrix

In [23]:
X_train_vec = X_train_vec.reshape(X_train_vec.shape[0], 1, X_train_vec.shape[1])

In [24]:
X_val_vec = matrix_val

In [25]:
X_val_vec = X_val_vec.reshape(X_val_vec.shape[0], 1, X_val_vec.shape[1])

In [26]:
print(X_train_vec)
print(X_train_vec.shape)

[[[ 0.54991382  0.14493077  0.06994156 ... -0.04249781 -0.1856176
   -0.03713015]]

 [[ 0.12405957  0.0848012  -0.02280761 ... -0.06037292 -0.01156013
    0.27493739]]

 [[ 0.17099953 -0.08276331  0.14694567 ... -0.1271266  -0.10794874
   -0.26167229]]

 ...

 [[-0.01771358  0.24970101 -0.15886398 ... -0.1035225   0.05642457
   -0.11164612]]

 [[ 0.0084172   0.11862163 -0.01767736 ...  0.07322099  0.04701766
   -0.14379349]]

 [[-0.01478438  0.04485109 -0.15111421 ... -0.25449198  0.05221124
   -0.1677798 ]]]
(10000, 1, 300)


In [41]:
def build_model(x_train):
    model = tf.keras.Sequential()
    input_shape = x_train.shape
    model.add(Dense(16, activation="relu", input_shape=input_shape[1:]))
    model.add(MaxPooling1D(1))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

In [42]:
# MAE : the average absolute distance between the predicted and target values
def compile_model(model):
    mae = tf.keras.losses.MeanAbsolutePercentageError()
    model.compile(loss=mae,
                  optimizer='adam',
                  metrics=['mean_squared_error', 'mean_absolute_error']
                  )
    return  model

In [43]:
def fit_model(x_train, y_train, x_val, y_val, model, batch_size,  epochs = 5):
    
    print('Train...')
    os.makedirs("./logs/CNN_logs",exist_ok=True)
    tensorboard = TensorBoard(log_dir=os.path.join('./logs/CNN_logs'), histogram_freq=0,
                                  write_graph=True, write_images=False,profile_batch = 100000000)

    # This callback will stop the training when there is no improvement in
    # the validation loss for 2 consecutive epochs.
    Es= EarlyStopping(monitor='loss', patience=2)
    
    callbacks = [Es, tensorboard]

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_val, y_val),
              callbacks= callbacks)
    return model

In [44]:
with tf.device("/CPU:0"):
    model = build_model(X_train_vec)
    model.summary()
    model = compile_model(model)
    model = fit_model(X_train_vec, y_train, X_val_vec, y_val, model, batch_size=200, epochs=3)

TypeError: The added layer must be an instance of class Layer. Found: <keras.layers.core.Dense object at 0x7fedcf1b9410>

## Save model 

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('./saved_model/CNN_model_FT')

In [ ]:
loss, mean_squared_error, mean_ab = model.evaluate(x_val_pad, y_val)

In [ ]:
# © Laëtitia CONSTANTIN & Axel CHENU 2021